# Neural networks simulation (Synchronization Problem)

This file is going to study any neural netwrok class which is defined in the `<network_reference.py>` file.

In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas() #This line make sure that the progress bars looks natural

## Order your neuron model of interest

In [3]:
current_models = ['IF','Rotational']
neuron_model = current_models[1]

In [4]:
with open("network_reference.py") as net_ref: 
    lines = net_ref.readlines() #read 
 
    #modify 
    lines[0] = "neuron_engine = '{}'\n".format(neuron_model) #you can replace zero with any line number. 
 
with open("network_reference.py", "w") as net_ref: 
    net_ref.writelines(lines) #write back 

In [5]:
from network_reference import Network_of_neurons

## Parameters and constants

In [6]:
num_neurons = 10000
total_time = 1000

num_ensembles_each_g = 2

In [7]:
random_input_span = (9.5,13.5)

In [8]:
connectivity_min,connectivity_max, connectivity_step =10, 20, 1
connectivity_span = np.arange(connectivity_min,connectivity_max,connectivity_step).round(2)

In [9]:
delay_min,delay_max, delay_step =0.05, 0.30, 0.02 #think what option serves us best
delay_span = np.arange(delay_min, delay_max, delay_step).round(2)

## Prepare the storage to record the outputs

In [10]:
current_size_networks_path = os.path.join(neuron_model + '_ensembles','N{}_T{}_I{}_{}'.format(num_neurons,total_time,random_input_span[0],random_input_span[1]) )
try:
    os.makedirs(current_size_networks_path)
except:
    pass

In [11]:
from storage_modifications import *

## Creating various ensembles:

In [12]:
for i,connectivity in enumerate(tqdm( connectivity_span , desc='Variation of connectivities') ):
    group_folder_name = 'g_{}'.format(connectivity)
    group_storage = make_inner_dir(current_size_networks_path, group_folder_name)
    
    for j,delay in enumerate(tqdm( delay_span , desc='Variation of delay') ):
        subgroup_folder_name = 'd_{}'.format(delay)
        subgroup_storage = make_inner_dir(group_storage, subgroup_folder_name)
        last_index = last_ensemble_index_in_group(subgroup_storage)

        for k in tqdm( range(num_ensembles_each_g) , desc='Making ensembles'):
            sample_model = Network_of_neurons(num_neurons=num_neurons,g=connectivity)
            sample_model.ignite(random_input_span, total_time = total_time,
                                time_step = 0.01, delay_time = delay)

            current_ensemble_folder = make_inner_dir(subgroup_storage, str(last_index + k + 1) )
            
            save_sigma(sample_model.report_sigma(), current_ensemble_folder)
            save_field_period(sample_model.report_e_period(), current_ensemble_folder)
            if neuron_model == current_models[1]: save_amin_saman_param(sample_model.report_sync_parameter(), current_ensemble_folder)
            save_mean_spiking_periods( sample_model.report_spikies_period_mean(), current_ensemble_folder )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


network dynamic:   0%|                                                                       | 0/99999 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
network dynamic: 100%|█████████████████████████████████████████████████████████| 99999/99999 [01:08<00:00, 1461.54it/s]


network dynamic:  90%|████████████████████████████████████████████████████▏     | 89992/99999 [01:07<00:11, 869.73it/s]

KeyboardInterrupt: 